<a href="https://colab.research.google.com/github/jsansao/teic-20231/blob/main/TEIC_Licao28bis_B2w_OutrosMetodos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Célula 1: Instalações e Importações
!pip install fasttext

import re
import string
import pandas as pd
import numpy as np

# Para pré-processamento
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

# Para divisão dos dados
from sklearn.model_selection import train_test_split

# Para Modelos 1 e 2 (sklearn)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score
from sklearn.decomposition import TruncatedSVD

# Para Modelo 3 (FastText)
import fasttext

# Para formatação
from IPython.display import display, Markdown

display(Markdown("## 1. Carregamento e Pré-processamento dos Dados (B2W-Reviews)"))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-3.0.1-py3-none-any.whl.metadata (10.0 kB)
Using cached pybind11-3.0.1-py3-none-any.whl (293 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp312-cp312-linux_x86_64.whl size=4498214 sha256=0052edc6cc794299ccc1527fced834f2847cf71fd92fdf7d73eb1f6c1e2219c9
  Stored in directory: /root/.cache/pip/wheels/20/27/95/a7baf1b435f1cbde017cabdf1e9688526d2b0e929255a359c6
Successfully built fasttext


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## 1. Carregamento e Pré-processamento dos Dados (B2W-Reviews)

In [3]:
!git clone https://github.com/americanas-tech/b2w-reviews01.git

Cloning into 'b2w-reviews01'...
remote: Enumerating objects: 10, done.
remote: Total 10 (delta 0), reused 0 (delta 0), pack-reused 10 (from 1)
Receiving objects: 100% (10/10), 19.82 MiB | 20.80 MiB/s, done.


In [5]:
# Célula 2: Carregar Dados da B2W-Reviews de uma URL

def carregar_dados_b2w():
    """Carrega o dataset B2W-Reviews a partir de uma URL e trata os labels."""

    # URL de um Gist com o dataset (pode levar um momento para carregar)
    url = "b2w-reviews01/B2W-Reviews01.csv"

    print(f"Carregando dados de: {url}")
    df = pd.read_csv(url, sep=',')

    # Remove colunas desnecessárias e linhas com dados faltantes
    df = df[['review_text', 'overall_rating']].dropna()

    # --- Mapeamento de Labels (Rating -> Sentimento) ---
    # 1. Remove reviews neutras (rating == 3)
    df_filtered = df.query("overall_rating != 3").copy()

    # 2. Mapeia 1-2 estrelas para 0 (Negativo) e 4-5 estrelas para 1 (Positivo)
    df_filtered['sentiment'] = df_filtered['overall_rating'].apply(lambda x: 1 if x > 3 else 0)

    # Seleciona apenas as colunas que vamos usar
    df_final = df_filtered[['review_text', 'sentiment']]

    return df_final

df = carregar_dados_b2w()

print("\nDados carregados e mapeados:")
print(f"Total de amostras: {len(df)}")
print("\nDistribuição de sentimentos:")
print(df['sentiment'].value_counts())

display(df.head())

Carregando dados de: b2w-reviews01/B2W-Reviews01.csv


/tmp/ipython-input-1693993881.py:10: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url, sep=',')



Dados carregados e mapeados:
Total de amostras: 113088

Distribuição de sentimentos:
sentiment
1    79316
0    33772
Name: count, dtype: int64


,review_text,sentiment
0,Estou contente com a compra entrega rápida o ú...,1
1,"Por apenas R$1994.20,eu consegui comprar esse ...",1
2,SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...,1
3,MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...,1
4,"A entrega foi no prazo, as americanas estão de...",1


In [6]:
# Célula 3: Função de Pré-processamento (Português)

# Obtém a lista de stopwords em PORTUGUÊS
stop_words = set(stopwords.words('portuguese'))

def preprocessar_texto(text):
    """
    Limpa o texto:
    1. Remove tags HTML (embora esta base não tenha muitas)
    2. Remove pontuação
    3. Converte para minúsculas
    4. Remove stopwords (em português)
    """
    if not isinstance(text, str):
        return ""

    # 1. Remove HTML
    text = re.sub(r'<[^>]+>', ' ', text)

    # 2. Remove pontuação
    text = text.translate(str.maketrans('', '', string.punctuation))

    # 3. Converte para minúsculas
    text = text.lower()

    # 4. Remove stopwords
    palavras = [palavra for palavra in text.split() if palavra not in stop_words]

    return ' '.join(palavras)

# Aplica o pré-processamento
# (Pode levar alguns minutos)
print("Iniciando pré-processamento...")
df['clean_text'] = df['review_text'].apply(preprocessar_texto)
print("Pré-processamento concluído.")

display(df.head())

Iniciando pré-processamento...
Pré-processamento concluído.


,review_text,sentiment,clean_text
0,Estou contente com a compra entrega rápida o ú...,1,contente compra entrega rápida único problema ...
1,"Por apenas R$1994.20,eu consegui comprar esse ...",1,apenas r199420eu consegui comprar lindo copo a...
2,SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...,1,supera agilidade praticidade outras panelas el...
3,MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...,1,filho amou parece verdade tantos detalhes têm
4,"A entrega foi no prazo, as americanas estão de...",1,entrega prazo americanas parabéns smart tv boa...


In [7]:
# Célula 4: Divisão de Treino e Teste

display(Markdown("## 2. Divisão de Treino e Teste"))

# Como este dataset não tem um split padrão, vamos criar um
# Usaremos 75% para treino e 25% para teste

X = df['clean_text']
y = df['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

print(f"Tamanho do treino: {len(X_train)} amostras")
print(f"Tamanho do teste: {len(X_test)} amostras")

## 2. Divisão de Treino e Teste

Tamanho do treino: 84816 amostras
Tamanho do teste: 28272 amostras


In [8]:
# Célula 5: Método 1 - TF-IDF com Regressão Logística

display(Markdown("## 3. Método 1: TF-IDF + Regressão Logística"))

# Cria um pipeline do Scikit-learn
pipeline_tfidf_lr = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=20000, ngram_range=(1, 2))),
    ('clf', LogisticRegression(solver='liblinear', C=10))
])

# Treina o modelo
print("Treinando TF-IDF + Regressão Logística...")
pipeline_tfidf_lr.fit(X_train, y_train)

# Faz previsões
y_pred_tfidf_lr = pipeline_tfidf_lr.predict(X_test)

# Avalia o modelo
print("\nResultados (TF-IDF + LR):")
report_tfidf_lr = classification_report(y_test, y_pred_tfidf_lr, output_dict=True)
print(classification_report(y_test, y_pred_tfidf_lr, target_names=['Negativo', 'Positivo']))

## 3. Método 1: TF-IDF + Regressão Logística

Treinando TF-IDF + Regressão Logística...

Resultados (TF-IDF + LR):
              precision    recall  f1-score   support

    Negativo       0.90      0.89      0.90      8443
    Positivo       0.96      0.96      0.96     19829

    accuracy                           0.94     28272
   macro avg       0.93      0.93      0.93     28272
weighted avg       0.94      0.94      0.94     28272



In [9]:
# Célula 6: Método 2 - LSA (TF-IDF + SVD) com Regressão Logística

display(Markdown("## 4. Método 2: LSA (Latent Semantic Analysis) + Regressão Logística"))

N_COMPONENTS_LSA = 300

pipeline_lsa_lr = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=20000, ngram_range=(1, 1))), # LSA geralmente funciona melhor com unigramas
    ('svd', TruncatedSVD(n_components=N_COMPONENTS_LSA)),
    ('clf', LogisticRegression(solver='liblinear', C=1))
])

# Treina o modelo
print(f"Treinando LSA (SVD com {N_COMPONENTS_LSA} componentes) + Regressão Logística...")
pipeline_lsa_lr.fit(X_train, y_train)

# Faz previsões
y_pred_lsa_lr = pipeline_lsa_lr.predict(X_test)

# Avalia o modelo
print("\nResultados (LSA + LR):")
report_lsa_lr = classification_report(y_test, y_pred_lsa_lr, output_dict=True)
print(classification_report(y_test, y_pred_lsa_lr, target_names=['Negativo', 'Positivo']))

## 4. Método 2: LSA (Latent Semantic Analysis) + Regressão Logística

Treinando LSA (SVD com 300 componentes) + Regressão Logística...

Resultados (LSA + LR):
              precision    recall  f1-score   support

    Negativo       0.87      0.85      0.86      8443
    Positivo       0.94      0.95      0.94     19829

    accuracy                           0.92     28272
   macro avg       0.90      0.90      0.90     28272
weighted avg       0.92      0.92      0.92     28272



In [10]:
# Célula 7: Método 3 - FastText

display(Markdown("## 5. Método 3: Classificador Nativo FastText"))

# O FastText requer um formato de arquivo específico:
# __label__<valor> <texto>

# Função para formatar e salvar os dados
# Desta vez, vamos salvar a partir das listas de X e y
def salvar_formato_fasttext(X_data, y_data, nome_arquivo):
    with open(nome_arquivo, 'w') as f:
        # y_data é um pd.Series, precisamos do índice para alinhar
        for index, text in X_data.items():
            label = y_data[index]
            linha = f"__label__{label} {text}\n"
            f.write(linha)

# Salva os arquivos de treino e teste
caminho_treino_ft = 'fasttext_b2w.train'
caminho_teste_ft = 'fasttext_b2w.test'

print("Salvando arquivos no formato FastText...")
salvar_formato_fasttext(X_train, y_train, caminho_treino_ft)
salvar_formato_fasttext(X_test, y_test, caminho_teste_ft)
print(f"Arquivos salvos em: {caminho_treino_ft} e {caminho_teste_ft}")

# Treina o modelo FastText
print("\nTreinando FastText...")
modelo_fasttext = fasttext.train_supervised(
    input=caminho_treino_ft,
    epoch=10,
    lr=0.5,
    wordNgrams=2,
    dim=100
)
print("Treinamento concluído.")

# Avalia o modelo no arquivo de teste
metricas_ft = modelo_fasttext.test(caminho_teste_ft)
acuracia_ft = metricas_ft[1]
print(f"\nAcurácia (nativa) do FastText: {acuracia_ft:.4f}")

# Para um relatório de classificação completo (comparável ao sklearn):
print("Calculando relatório de classificação completo para FastText...")
textos_teste = X_test.tolist()

# model.predict() retorna o label (ex: '__label__1') e a confiança
pred_labels_ft_raw = modelo_fasttext.predict(textos_teste)[0]

# Limpa o '__label__' para obter '0' ou '1' e converte para int
y_pred_fasttext = [int(label[0].replace('__label__', '')) for label in pred_labels_ft_raw]

# Avalia o modelo
print("\nResultados (FastText):")
report_fasttext = classification_report(y_test, y_pred_fasttext, output_dict=True)
print(classification_report(y_test, y_pred_fasttext, target_names=['Negativo', 'Positivo']))

## 5. Método 3: Classificador Nativo FastText

Salvando arquivos no formato FastText...
Arquivos salvos em: fasttext_b2w.train e fasttext_b2w.test

Treinando FastText...
Treinamento concluído.

Acurácia (nativa) do FastText: 0.9380
Calculando relatório de classificação completo para FastText...

Resultados (FastText):
              precision    recall  f1-score   support

    Negativo       0.89      0.90      0.90      8443
    Positivo       0.96      0.95      0.96     19829

    accuracy                           0.94     28272
   macro avg       0.92      0.93      0.93     28272
weighted avg       0.94      0.94      0.94     28272



In [11]:
# Célula 8: Comparação Final dos Resultados

display(Markdown("## 6. Comparação Final (B2W-Reviews)"))

# Coleta as métricas principais (macro avg) de cada relatório
def extrair_metricas(report, model_name):
    return {
        'Modelo': model_name,
        'Acurácia': report['accuracy'],
        'Precisão (Macro)': report['macro avg']['precision'],
        'Recall (Macro)': report['macro avg']['recall'],
        'F1-Score (Macro)': report['macro avg']['f1-score']
    }

metricas = [
    extrair_metricas(report_tfidf_lr, 'TF-IDF + LR'),
    extrair_metricas(report_lsa_lr, 'LSA + LR'),
    extrair_metricas(report_fasttext, 'FastText')
]

# Cria um DataFrame para fácil visualização
df_resultados = pd.DataFrame(metricas)
df_resultados = df_resultados.set_index('Modelo')

display(df_resultados.style.format("{:.4f}"))

display(Markdown(
"""
### Observações (Esperadas):

* **TF-IDF + LR:** Novamente, deve ser um *baseline* muito forte.
* **LSA + LR:** É provável que o LSA tenha uma performance um pouco inferior ao TF-IDF puro, pois a compressão de dimensão pode perder nuances.
* **FastText:** Espera-se que o FastText tenha o melhor desempenho. O texto em português (especialmente em *reviews*) contém muitas gírias, abreviações e erros de digitação. A capacidade do FastText de usar n-gramas de caracteres (informação de sub-palavras) é uma vantagem imensa nesse cenário.
"""
))

## 6. Comparação Final (B2W-Reviews)

,Acurácia,Precisão (Macro),Recall (Macro),F1-Score (Macro)
Modelo,,,,
TF-IDF + LR,0.9398,0.9293,0.9267,0.9280
LSA + LR,0.9180,0.9048,0.8981,0.9013
FastText,0.9380,0.9244,0.9283,0.9263



### Observações (Esperadas):

* **TF-IDF + LR:** Novamente, deve ser um *baseline* muito forte.
* **LSA + LR:** É provável que o LSA tenha uma performance um pouco inferior ao TF-IDF puro, pois a compressão de dimensão pode perder nuances.
* **FastText:** Espera-se que o FastText tenha o melhor desempenho. O texto em português (especialmente em *reviews*) contém muitas gírias, abreviações e erros de digitação. A capacidade do FastText de usar n-gramas de caracteres (informação de sub-palavras) é uma vantagem imensa nesse cenário.
